In [1]:
import pandas as pd
from collections import Counter
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\results.csv")
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [9]:
#df.tournament.unique()

In [10]:
dfFIFAq = df.loc[df.tournament == 'FIFA World Cup qualification'].reset_index()

In [31]:
dfFIFA = df.loc[df.tournament == 'FIFA World Cup'].reset_index()
dfFIFA['Year'] = dfFIFA['date'].apply(lambda x: int(str.split(x,"-")[0]))

In [16]:
dfFIFAq.shape

(7363, 9)

In [11]:
dfFIFAq['Year'] = dfFIFAq['date'].apply(lambda x: int(str.split(x,"-")[0]))

In [12]:
#dfFIFA = dfFIFA.drop(columns = ['index','date','city','country','neutral'])
dfFIFAq = dfFIFAq.drop(columns = ['index','date','city','neutral'])
dfFIFAq['home_games'] = 1
dfFIFAq['away_games'] = 1
dfFIFAq.head()

,home_team,away_team,home_score,away_score,tournament,country,Year,home_games,away_games
0,Sweden,Estonia,6,2,FIFA World Cup qualification,Sweden,1933,1,1
1,Lithuania,Sweden,0,2,FIFA World Cup qualification,Lithuania,1933,1,1
2,Yugoslavia,Switzerland,2,2,FIFA World Cup qualification,Yugoslavia,1933,1,1
3,Poland,Czechoslovakia,1,2,FIFA World Cup qualification,Poland,1933,1,1
4,Switzerland,Romania,2,2,FIFA World Cup qualification,Switzerland,1933,1,1


In [13]:
#dfFIFACountry = dfFIFA[['Year','country']]
#dfFIFACountry = dfFIFACountry.drop_duplicates().reset_index().drop(columns = ['index'])
#dfFIFACountry

In [25]:
#dfFIFA.loc[(dfFIFA.home_team=='Chile') & (dfFIFA.Year == 1930)]

In [14]:
dfFIFAq_home = dfFIFAq.groupby(['Year','home_team']).sum().rename(columns = {'away_score':'home_goals_conceded'}).drop(columns ='away_games') #.reset_index().set_index('Year')
dfFIFAq_home.head()

home_score  home_goals_conceded  home_games
Year home_team                                               
1933 Lithuania             0                    2           1
     Poland                1                    2           1
     Sweden                6                    2           1
     Switzerland           2                    2           1
     Yugoslavia            2                    2           1

In [15]:
dfFIFAq_away = dfFIFAq.groupby(['Year','away_team']).sum().rename(columns = {'home_score':'away_goals_conceded'}).drop(columns = ['home_games'])#.reset_index().set_index('Year')
dfFIFAq_away.head()

away_goals_conceded  away_score  away_games
Year away_team                                                  
1933 Czechoslovakia                    1           2           1
     Estonia                           6           2           1
     Romania                           2           2           1
     Sweden                            0           2           1
     Switzerland                       2           2           1

In [28]:
dfFIFA_away_ri = dfFIFA_away.reset_index()
dfFIFA_away_ri.loc[dfFIFA_away_ri.Year == 1930]

,Year,away_team,away_goals_conceded,away_score,away_games
0,1930,Argentina,4,2,1
1,1930,Brazil,0,4,1
2,1930,Chile,3,1,1
3,1930,France,2,0,2
4,1930,Mexico,13,4,3
5,1930,Paraguay,0,1,1
6,1930,Peru,1,0,1
7,1930,Romania,5,3,2
8,1930,United States,6,7,3
9,1930,Yugoslavia,7,7,3


In [18]:
dfFIFAqdata = pd.concat([dfFIFAq_away, dfFIFAq_home.reindex(dfFIFAq_away.index)], axis=1).reset_index().set_index('Year').rename(columns = {'away_team':'team'})
dfFIFAqdata = dfFIFAqdata.fillna(0)
dfFIFAqdata.head()

,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games
Year,,,,,,,
1933,Czechoslovakia,1,2,1,0.0,0.0,0.0
1933,Estonia,6,2,1,0.0,0.0,0.0
1933,Romania,2,2,1,0.0,0.0,0.0
1933,Sweden,0,2,1,6.0,2.0,1.0
1933,Switzerland,2,2,1,2.0,2.0,1.0


In [19]:
dfFIFAq_total_avg_1 = dfFIFAq.groupby(['Year']).mean().rename(columns = {'home_score':'hs_mean','away_score':'as_mean'}).drop(columns = ['home_games','away_games'])
dfFIFAq_total_avg_1.head()

,hs_mean,as_mean
Year,,
1933,2.200000,2.000000
1934,3.000000,2.750000
1937,3.000000,1.428571
1938,3.125000,1.125000
1949,3.227273,1.772727


In [20]:
dfFIFAq_total_avg_2 = dfFIFAq_total_avg_1.copy().rename(columns = {'hs_mean':'as_conceded_mean','as_mean':'hs_conceded_mean'})
dfFIFAq_total_avg_2.head()

,as_conceded_mean,hs_conceded_mean
Year,,
1933,2.200000,2.000000
1934,3.000000,2.750000
1937,3.000000,1.428571
1938,3.125000,1.125000
1949,3.227273,1.772727


In [21]:
dfFIFAq_total_avg = pd.concat([dfFIFAq_total_avg_1, dfFIFAq_total_avg_2], axis=1, join='inner')
dfFIFAq_total_avg.head()

,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
Year,,,,
1933,2.200000,2.000000,2.200000,2.000000
1934,3.000000,2.750000,3.000000,2.750000
1937,3.000000,1.428571,3.000000,1.428571
1938,3.125000,1.125000,3.125000,1.125000
1949,3.227273,1.772727,3.227273,1.772727


In [23]:
dfFIFAqdata = dfFIFAqdata.merge(dfFIFAq_total_avg, on='Year').reset_index()
dfFIFAqdata.head()

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
0,1933,Czechoslovakia,1,2,1,0.0,0.0,0.0,2.2,2.0,2.2,2.0
1,1933,Estonia,6,2,1,0.0,0.0,0.0,2.2,2.0,2.2,2.0
2,1933,Romania,2,2,1,0.0,0.0,0.0,2.2,2.0,2.2,2.0
3,1933,Sweden,0,2,1,6.0,2.0,1.0,2.2,2.0,2.2,2.0
4,1933,Switzerland,2,2,1,2.0,2.0,1.0,2.2,2.0,2.2,2.0


In [24]:
dfFIFAqdata['H_AttackS'] = (dfFIFAqdata['home_score']/dfFIFAqdata['home_games'])/dfFIFAqdata['hs_mean']
dfFIFAqdata['A_AttackS'] = (dfFIFAqdata['away_score']/dfFIFAqdata['away_games'])/dfFIFAqdata['as_mean']
dfFIFAqdata['H_DefenseS'] = (dfFIFAqdata['home_goals_conceded']/dfFIFAqdata['home_games'])/dfFIFAqdata['hs_conceded_mean']
dfFIFAqdata['A_DefenseS'] = (dfFIFAqdata['away_goals_conceded']/dfFIFAqdata['away_games'])/dfFIFAqdata['as_conceded_mean']

In [38]:
#dfFIFAqdata = dfFIFAqdata.set_index(['Year','team'])
dfFIFAqdata.head()
dfFIFAqdata = dfFIFAqdata.reset_index()

In [89]:
dfFIFAqdata

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS,FIFAyear
0,1933,Czechoslovakia,1,2,1,0.0,0.0,0.0,2.200000,2.00000,2.200000,2.00000,NaN,1.000000,NaN,0.454545,1934.0
1,1933,Estonia,6,2,1,0.0,0.0,0.0,2.200000,2.00000,2.200000,2.00000,NaN,1.000000,NaN,2.727273,1934.0
2,1933,Romania,2,2,1,0.0,0.0,0.0,2.200000,2.00000,2.200000,2.00000,NaN,1.000000,NaN,0.909091,1934.0
3,1933,Sweden,0,2,1,6.0,2.0,1.0,2.200000,2.00000,2.200000,2.00000,2.727273,1.000000,1.000000,0.000000,1934.0
4,1933,Switzerland,2,2,1,2.0,2.0,1.0,2.200000,2.00000,2.200000,2.00000,0.909091,1.000000,1.000000,0.909091,1934.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,2021,Turkey,0,3,1,7.0,5.0,2.0,1.820755,1.54717,1.820755,1.54717,1.922280,1.939024,1.615854,0.000000,NaN
3322,2021,Turks and Caicos Islands,5,0,1,0.0,7.0,1.0,1.820755,1.54717,1.820755,1.54717,0.000000,0.000000,4.524390,2.746114,NaN
3323,2021,U.S. Virgin Islands,1,0,1,0.0,3.0,1.0,1.820755,1.54717,1.820755,1.54717,0.000000,0.000000,1.939024,0.549223,NaN
3324,2021,Ukraine,1,1,1,2.0,2.0,2.0,1.820755,1.54717,1.820755,1.54717,0.549223,0.646341,0.646341,0.549223,NaN


In [90]:
dfFIFAqdata_wNa = dfFIFAqdata.fillna(0)

In [83]:
# w -> without
#dfFIFAqdata_wNa

In [92]:
#dfFIFAqdata_wNa.reset_index().to_csv(r"D:\Semester 7\Machine Learning\Assignment\DataFIFAQualifications_stat.csv")

In [86]:
yearlist = dfFIFA.Year.unique().tolist()
#yearlist.sort(reverse=True)
yearlist

[1930,
 1934,
 1938,
 1950,
 1954,
 1958,
 1962,
 1966,
 1970,
 1974,
 1978,
 1982,
 1986,
 1990,
 1994,
 1998,
 2002,
 2006,
 2010,
 2014,
 2018]

In [87]:
def fifaYear(dfFIFAqdata):
    for year in yearlist:
        print(year)
        if (dfFIFAqdata['Year']<=year):
            return year

In [88]:
dfFIFAqdata['FIFAyear'] = dfFIFAqdata.apply(fifaYear,axis = 1)

1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1930
1934
1938
1950
1954
1930
1934
1938
1950
1954
1930
1934
1938
1950
1954
1930
1934
1938
1950
1954
1930
1934
1938
1950
1954
1930
1934
1938
1950
1954


1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1930
1934
1938
1950
1954
1958
1962
1966
1970


1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1930
1934


1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974


1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986


...

1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966


2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002


2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966


2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966


1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002
2006
2010
2014
2018
1930
1934
1938
1950
1954
1958
1962
1966
1970
1974
1978
1982
1986
1990
1994
1998
2002


In [93]:
contdic={}

for year in yearlist:
    #print(year)
    wonlist = []
    for index,row in dfFIFA.iterrows():
        if (row['Year']<=year):
            if row['Year'] == year:
                if row['home_score']>row['away_score']:
                    wonlist.append(row['home_team'])
                elif row['home_score']<row['away_score']:
                    wonlist.append(row['away_team'])
        contdic[year] = wonlist

In [94]:
k = {}
for year in yearlist:
    k[year] = Counter(contdic[year])

In [95]:
# to make a df with countries and corrosponding number of wins
dflist = []
for year in yearlist:
    df = pd.DataFrame.from_dict(k[year],orient = 'index').reset_index().rename(columns ={'index':'team',0:'WinCount'} )
    df['Year'] = year
    # this is a list of data frames
    dflist.append(df)


In [97]:
result = dflist[0].append(dflist[1:]).set_index(['Year','team'])
result

WinCount
Year team                   
1930 United States         2
     France                1
     Yugoslavia            2
     Romania               1
     Argentina             4
...                      ...
2018 Peru                  1
     Argentina             1
     South Korea           1
     Poland                1
     Tunisia               1

[322 rows x 1 columns]

In [41]:
pd.merge(dfFIFAdata, result, left_index=True, right_index=True)

,,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS,WinCount
Year,team,,,,,,,,,,,,,,,
1986,Germany,6,6,6,2.0,1.0,1.0,1.461538,1.076923,1.461538,1.076923,1.368421,0.928571,0.928571,0.684211,3
1934,Spain,3,4,3,0.0,0.0,0.0,2.235294,1.882353,2.235294,1.882353,NaN,0.708333,NaN,0.447368,1
1982,Russia,2,2,3,5.0,2.0,2.0,1.826923,0.980769,1.826923,0.980769,1.368421,0.679739,1.019608,0.364912,2
1950,Switzerland,3,4,2,0.0,3.0,1.0,1.954545,2.045455,1.954545,2.045455,0.000000,0.977778,1.466667,0.767442,1
1990,Argentina,2,2,4,3.0,2.0,3.0,1.269231,0.942308,1.269231,0.942308,0.787879,0.530612,0.707483,0.393939,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,Switzerland,0,2,2,2.0,0.0,2.0,1.390625,0.906250,1.390625,0.906250,0.719101,1.103448,0.000000,0.000000,2
1930,United States,6,7,3,0.0,0.0,0.0,2.277778,1.611111,2.277778,1.611111,NaN,1.448276,NaN,0.878049,2
1978,Tunisia,2,3,3,0.0,0.0,0.0,1.473684,1.210526,1.473684,1.210526,NaN,0.826087,NaN,0.452381,1


In [42]:
dfFIFAdata_final = dfFIFAdata.join(result)

In [43]:
dfFIFAdata_final = dfFIFAdata_final.fillna(0).reset_index()
dfFIFAdata_final.head()

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS,WinCount
0,1930,Argentina,4,2,1,16.0,5.0,4.0,2.277778,1.611111,2.277778,1.611111,1.756098,1.241379,0.775862,1.756098,4.0
1,1930,Brazil,0,4,1,1.0,2.0,1.0,2.277778,1.611111,2.277778,1.611111,0.439024,2.482759,1.241379,0.000000,1.0
2,1930,Chile,3,1,1,4.0,0.0,2.0,2.277778,1.611111,2.277778,1.611111,0.878049,0.620690,0.000000,1.317073,2.0
3,1930,France,2,0,2,4.0,1.0,1.0,2.277778,1.611111,2.277778,1.611111,1.756098,0.000000,0.620690,0.439024,1.0
4,1930,Mexico,13,4,3,0.0,0.0,0.0,2.277778,1.611111,2.277778,1.611111,0.000000,0.827586,0.000000,1.902439,0.0


In [91]:
#dfFIFAdata_final.to_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\dfFIFAdata_final.csv")

In [46]:
dfFIFAdata_final = dfFIFAdata_final.merge(dfFIFACountry, on = 'Year')

In [47]:
#dfFIFAdata_final.to_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\dfFIFAdata_final_wcountry.csv")